# TensorBoard 測試

In [1]:
# 刪除 log 目錄
!rd .\logs /S /Q 

In [2]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist


# 匯入 MNIST 手寫阿拉伯數字 訓練資料
(x_train, y_train),(x_test, y_test) = mnist.load_data()


# 訓練/測試資料的 X/y 維度
x_train.shape, y_train.shape,x_test.shape, y_test.shape

# 特徵縮放，使用常態化(Normalization)，公式 = (x - min) / (max - min)
# 顏色範圍：0~255，所以，公式簡化為 x / 255
# 注意，顏色0為白色，與RGB顏色不同，(0,0,0) 為黑色。
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0

In [3]:
# 建立模型
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])


In [4]:
# 設定優化器(optimizer)、損失函數(loss)、效能衡量指標(metrics)的類別
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define 訓練及測試的效能衡量指標(Metrics)
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [5]:
def train_step(model, optimizer, x_train, y_train):
    # 自動微分
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # 計算訓練的效能衡量指標
    train_loss(loss)
    train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
    # 預測
    predictions = model(x_test)
    # 計算損失
    loss = loss_object(y_test, predictions)

    # 計算測試的效能衡量指標
    test_loss(loss)
    test_accuracy(y_test, predictions)

In [6]:
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# 指定訓練的 log 檔名
train_log_dir = '.\\logs\\gradient_tape\\' + current_time + '\\train'
# 指定測試的 log 檔名
test_log_dir = '.\\logs\\gradient_tape\\' + current_time + '\\test'

# 開啟檔案
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [7]:
# 將訓練/測試資料轉成 Tensorflow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# 每次從 60000 筆訓練資料隨機抽出 64 筆
train_dataset = train_dataset.shuffle(60000).batch(64)
# 每次從 10000 筆測試資料隨機抽出 64 筆
test_dataset = test_dataset.batch(64)

In [8]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_dataset:
        train_step(model, optimizer, x_train, y_train)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for (x_test, y_test) in test_dataset:
        test_step(model, x_test, y_test)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
    
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                                                 train_loss.result(), 
                                                 train_accuracy.result()*100,
                                                 test_loss.result(), 
                                                 test_accuracy.result()*100))

    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()


Epoch 1, Loss: 7.301743030548096, Accuracy: 54.3466682434082, Test Loss: 5.807878494262695, Test Accuracy: 63.810001373291016
Epoch 2, Loss: 5.913661479949951, Accuracy: 63.150001525878906, Test Loss: 5.674729347229004, Test Accuracy: 64.6300048828125
Epoch 3, Loss: 5.791152477264404, Accuracy: 63.936668395996094, Test Loss: 5.490237236022949, Test Accuracy: 65.8699951171875
Epoch 4, Loss: 5.6589274406433105, Accuracy: 64.80500030517578, Test Loss: 5.425393104553223, Test Accuracy: 66.20999908447266
Epoch 5, Loss: 5.597524642944336, Accuracy: 65.16500091552734, Test Loss: 5.283638954162598, Test Accuracy: 67.07999420166016


In [9]:
# 載入 TensorBoard notebook extension，即可在 jupyter notebook 啟動 Tensorboard
%load_ext tensorboard

In [17]:
# 啟動 Tensorboard
%tensorboard --logdir logs/gradient_tape

Reusing TensorBoard on port 6006 (pid 7044), started 0:12:00 ago. (Use '!kill 7044' to kill it.)

## 使用瀏覽器輸入以下網址，即可觀看訓練資訊：
## http://localhost:6006/

In [11]:
!taskkill /IM "tensorboard.exe" /F
# or 
# !taskkill /F /PID 15784

���\: �B�z�{�� "tensorboard.exe" (PID 18624) �w�g�פ�F�C


## 寫入圖片

In [13]:
# 任意找一張圖片
img = x_train[0].numpy().reshape((-1, 28, 28, 1))
img.shape

(1, 28, 28, 1)

In [14]:
# 指定 log 檔名
logdir = ".\\logs\\train_data\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the reshaped image.
with file_writer.as_default():
    # 將圖片寫入 log 檔
    tf.summary.image("Training data", img, step=0)

In [16]:
%tensorboard --logdir logs/train_data

Reusing TensorBoard on port 6006 (pid 18528), started 0:10:44 ago. (Use '!kill 18528' to kill it.)